Github Link: https://github.com/Rezela/Natural-Language-Processing-2

## Assignment 2: Probabilistic Models and Vector Space Applications

### Assignment Overview

This assignment builds on fundamental text processing techniques to explore probabilistic language modeling, text classification, and the practical application of vector space models for information retrieval. You will implement a simple n-gram language model, build a complete text classification pipeline, develop a search engine, and analyze the core components of sequence models.

You are required to complete five coding-related tasks. For each task, you will be working with a specified dataset or corpus. Please submit your solutions in this single Jupyter Notebook (`.ipynb`) file, clearly marking each task. Ensure your code is well-commented and your findings are explained in markdown cells where requested.

### Task 1: Implementing a Bigram Language Model with Laplace Smoothing (20 Marks)

**Objective:** To understand the fundamentals of n-gram language models, including probability calculation, smoothing, and evaluation with perplexity.

**Description:** You will implement a Bigram language model from scratch. Your model will be trained on a small corpus and will use Add-One (Laplace) smoothing to handle unseen n-grams.

**Your task is to:**

1.  **Implement a training function `train_bigram_model(corpus)`:**
    * The corpus will be a list of sentences, where each sentence is a list of tokens.
    * The function should count all unigrams and bigrams in the corpus.
    * It should return the unigram counts, bigram counts, and the vocabulary size (V).

2.  **Implement a probability function `calculate_bigram_prob(prev_word, word, unigram_counts, bigram_counts, V)`:**
    * This function should calculate the smoothed probability of a `word` given the `prev_word` using the formula for Laplace (Add-One) smoothing: $P(w_i | w_{i-1}) = \frac{C(w_{i-1}, w_i) + 1}{C(w_{i-1}) + V}$.

3.  **Implement a perplexity calculation function `calculate_perplexity(sentence, ...)`:**
    * This function should take a test sentence and your trained model components as input.
    * It should calculate the perplexity of the sentence using the formula: $PP(W) = P(w_1, w_2, ..., w_N)^{-1/N}$. Remember to handle the start of the sentence appropriately (e.g., by assuming a start token `<S>`).

4.  **Train and Evaluate:**
    * Train your model on the provided `train_corpus`.
    * Calculate and print the perplexity of your model on the `test_sentence`.

**Corpus:**

```python
# Sample corpus for training and testing
train_corpus = [["<S>", "i", "am", "sam", "</S>"], ["<S>", "sam", "i", "am", "</S>"], ["<S>", "i", "do", "not", "like", "green", "eggs", "and", "ham", "</S>"]]
test_sentence = ["<S>", "i", "like", "green", "ham", "</S>"]
```

In [2]:
import urllib.request

# Your code for Task 1 here
import numpy as np
from collections import Counter, defaultdict

# Provided Corpus
train_corpus = [["<S>", "i", "am", "sam", "</S>"], ["<S>", "sam", "i", "am", "</S>"], ["<S>", "i", "do", "not", "like", "green", "eggs", "and", "ham", "</S>"]]
test_sentence = ["<S>", "i", "like", "green", "ham", "</S>"]

def train_bigram_model(corpus):
    unigram_counts = Counter()  # 计数器
    bigram_counts = Counter()
    vocab = set()  # Vocabulary set集合 自动去重

    for sentence in corpus:
        for i in range(len(sentence)):
            unigram_counts[sentence[i]]+=1
            vocab.add(sentence[i])
            # if not first word -> bigram
            if i > 0:
                bigram = (sentence[i-1], sentence[i])
                bigram_counts[bigram] += 1
    vocab_size = len(vocab)
    return unigram_counts, bigram_counts, vocab_size

# Calculate smoothed probability for one bigram(given 1 prev_word and 1 current word)
def calculate_bigram_prob(prev_word, word, unigram_counts, bigram_counts, V):
    # Implement smoothed probability calculation
    bigram = (prev_word, word)
    bigram_count = bigram_counts[bigram]
    unigram_count = unigram_counts[prev_word]
    # Laplace (Add-One) smoothing
    prob = (bigram_count + 1) / (unigram_count + V)
    return prob

def calculate_perplexity(sentence, unigram_counts, bigram_counts, V):
    # Implement perplexity calculation
    '''
    first: logPP(W) = (-1/N)*sum(log(p(w i|w i-1)))
    then: PP(W) = exp(logPP(w))
    '''
    N = len(sentence) - 1
    sum_prob = 0
    for i in range(N):
        bigram = (sentence[i],sentence[i+1])
        bigram_prob = calculate_bigram_prob(bigram[0], bigram[1], unigram_counts, bigram_counts, V)
        sum_prob += np.log(bigram_prob)
    perplexity = np.exp(-sum_prob/N)
    return perplexity

if __name__ == '__main__':
    # Train the model
    unigram_counts, bigram_counts, V = train_bigram_model(train_corpus)

    # Calculate and print perplexity
    perplexity = calculate_perplexity(test_sentence, unigram_counts, bigram_counts, V)
    print(f"Perplexity of the test sentence: {perplexity:.2f}")


Perplexity of the test sentence: 8.37


### Task 1 Result:

Perplexity of the test sentence: **8.37**。


### Task 2: Text Classification with TF-IDF and Naive Bayes (20 Marks)

**Objective:** To build a complete text classification pipeline using TF-IDF feature extraction and a Multinomial Naive Bayes classifier.

**Description:** You will use `scikit-learn` to classify SMS messages as either "spam" or "ham" (not spam). This task integrates vector space representation with a classic probabilistic model.

**Your task is to:**

1.  Load the SMS Spam Collection dataset.
2.  Split the dataset into an 80% training set and a 20% testing set.
3.  Create a text processing pipeline using `sklearn.pipeline.Pipeline` that consists of two steps:
    * `TfidfVectorizer`: To convert text messages into TF-IDF vectors. Use the default parameters.
    * `MultinomialNB`: The Multinomial Naive Bayes classifier.
4.  Train the pipeline on the training data.
5.  Evaluate the trained model on the testing data. Print the following:
    * The accuracy of the model.
    * A full classification report (including precision, recall, and F1-score for each class) using `sklearn.metrics.classification_report`.
6.  Use the trained pipeline to predict the class of two new messages: `"Congratulations! You've won a $1,000 gift card. Go to http://example.com to claim now."` and `"Hi mom, I'll be home for dinner tonight."`

**Dataset:**

* **SMS Spam Collection Dataset:** A public set of SMS labeled messages.
* **Access:** Download from the UCI Machine Learning Repository: [SMS Spam Collection](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You will need the `SMSSpamCollection` file.

In [8]:
# Your code for Task 2 here
import os
import urllib.request
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset, e.g, as follows. But you may modify it.
# df = pd.read_csv('path_to_your_dataset/SMSSpamCollection', sep='\t', header=None, names=['label', 'message'])

# download dataset
dataset_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"
dataset_zip = "smsspamcollection.zip"
dataset_file = "SMSSpamCollection"

if not os.path.exists(dataset_file):
    print("Downloading dataset...")
    # download
    urllib.request.urlretrieve(dataset_url, dataset_zip)
    # unzip
    import zipfile
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall(".")  # unzip to current directory
    print("Dataset downloaded and extracted.")

# Load the dataset, Tab("\t") as separator, without a line of header, column 1: label, column 2: message
df = pd.read_csv(dataset_file, sep="\t", header=None, names=["label", "message"])
print("Dataset loaded. Shape:", df.shape)
print(df.head())

# Split data
x = df["message"]
y = df["label"]

# set a random seed: 28, stratify=y to make sure test size has same proportion of each class as the whole dataset
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=28, stratify=y
)

# Create and train the pipeline
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("nb", MultinomialNB())
])

# Train the pipeline
pipeline.fit(x_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(x_test)

print("Accuracy: ", accuracy_score(y_test, y_pred))
print("\nClassification report:\n", classification_report(y_test, y_pred))

# Predict on new messages
new_messages = [
    "Congratulations! You've won a $1,000 gift card. Go to http://example.com to claim now.",
    "Hi mom, I'll be home for dinner tonight."
]
predictions = pipeline.predict(new_messages)
for message, prediction in zip(new_messages, predictions):  # 两个列表并行配对遍历
    print(f"Message: {message}\nPrediction: {prediction}\n")

Dataset loaded. Shape: (5572, 2)
  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
Accuracy:  0.9623318385650225

Classification report:
               precision    recall  f1-score   support

         ham       0.96      1.00      0.98       966
        spam       1.00      0.72      0.84       149

    accuracy                           0.96      1115
   macro avg       0.98      0.86      0.91      1115
weighted avg       0.96      0.96      0.96      1115

Message: Congratulations! You've won a $1,000 gift card. Go to http://example.com to claim now.
Prediction: spam

Message: Hi mom, I'll be home for dinner tonight.
Prediction: ham



### Task 2 Result
### Dataset
Loaded dataset shape: **(5572, 2)**

| label | message |
|-------|---------|
| ham   | Go until jurong point, crazy.. Available only ... |
| ham   | Ok lar... Joking wif u oni... |
| spam  | Free entry in 2 a wkly comp to win FA Cup fina... |
| ham   | U dun say so early hor... U c already then say... |
| ham   | Nah I don't think he goes to usf, he lives aro... |

---

### Model Performance
**Accuracy:** `0.9623`

#### Classification Report

| Class | Precision | Recall | F1-score | Support |
|-------|-----------|--------|----------|---------|
| ham   | 0.96      | 1.00   | 0.98     | 966     |
| spam  | 1.00      | 0.72   | 0.84     | 149     |
| **Accuracy** |       |        | **0.96** | 1115    |
| **Macro avg** | 0.98 | 0.86   | 0.91     | 1115    |
| **Weighted avg** | 0.96 | 0.96 | 0.96     | 1115    |

---

### Predictions on New Messages
`"Congratulations! You've won a $1,000 gift card. Go to http://example.com to claim now."` : **spam**

`"Hi mom, I'll be home for dinner tonight."` : **ham**


### Task 3: Building a Simple Information Retrieval System (20 Marks)

**Objective:** To apply TF-IDF and Cosine Similarity to build a basic document retrieval system that ranks documents based on their relevance to a query.

**Description:** You will create a system that takes a text query and returns the most relevant documents from a small corpus. This is the core principle behind search engines.

**Your task is to:**

1.  Use the provided `document_corpus`.
2.  Create a `TfidfVectorizer` and fit it on the corpus to learn the vocabulary and IDF weights.
3.  Transform the corpus into a TF-IDF document-term matrix.
4.  Write a function `rank_documents(query, vectorizer, doc_term_matrix, top_n=3)` that:
    * Takes a `query` string, the fitted `vectorizer`, the document-term `matrix`, and an optional `top_n` integer.
    * Transforms the input query into a TF-IDF vector using the *same* vectorizer.
    * Calculates the cosine similarity between the query vector and all document vectors in the matrix.
    * Returns the indices and content of the `top_n` most similar documents.
5.  Demonstrate your system by running it with the query `"deep learning models for vision"` and printing the ranked results.

**Dataset:**

```python
# A small corpus of document abstracts
document_corpus = [
    "The field of machine learning has seen rapid growth in recent years, especially in deep learning.",
    "Natural language processing allows machines to understand and respond to human text.",
    "Computer vision focuses on enabling computers to see and interpret the visual world.",
    "Deep learning models like convolutional neural networks are powerful for computer vision tasks.",
    "Recurrent neural networks are often used for sequential data in natural language processing."
    ...
]
```

In [4]:
# Your code for Task 3 here
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

document_corpus = [
    "The field of machine learning has seen rapid growth in recent years, especially in deep learning.",
    "Natural language processing allows machines to understand and respond to human text.",
    "Computer vision focuses on enabling computers to see and interpret the visual world.",
    "Deep learning models like convolutional neural networks are powerful for computer vision tasks.",
    "Recurrent neural networks are often used for sequential data in natural language processing.",
    "The advances in reinforcement learning have led to breakthroughs in game playing and robotics.",
    "Transfer learning enables models trained on large datasets to be adapted for new tasks with limited data.",
    "Unsupervised learning techniques can discover hidden patterns in data without labeled examples.",
    "Optimization algorithms such as stochastic gradient descent are crucial for training neural networks.",
    "Attention mechanisms have improved the performance of natural language translation and image captioning.",
    "Generative adversarial networks create realistic images and are used for data augmentation.",
    "Feature engineering and selection are important steps in classical machine learning pipelines.",
    "Object detection is a key task in computer vision that involves locating instances within images.",
    "The combination of convolutional and recurrent networks is used for video classification tasks.",
    "Zero-shot learning allows models to recognize objects and concepts they have not seen during training.",
    "Natural language generation is used for creating text summaries and chatbot responses.",
    "Graph neural networks leverage graph structures for tasks such as social network analysis and chemistry.",
    "Hyperparameter tuning can significantly improve the accuracy of deep learning models.",
    "Cross-modal learning involves integrating information from multiple data sources such as text and images.",
    "Evaluating model performance requires a good choice of metrics such as F1-score and RMSE."
]

# Create and fit the vectorizer
vectorizer = TfidfVectorizer()

# Transform the corpus to DTM
doc_term_matrix = vectorizer.fit_transform(document_corpus)

def rank_documents(query, vectorizer, doc_term_matrix, top_n=3):
    vec_query = vectorizer.transform([query])
    similarities = cosine_similarity(vec_query, doc_term_matrix).flatten()

    # Get indices of top_n documents, sorted by similarity
    top_indices = similarities.argsort()[::-1][:top_n]

    results = []
    for rank, index in enumerate(top_indices, start=1):
        results.append((rank, index, document_corpus[index], similarities[index]))
    return results

# Demonstrate with a sample query
query = "deep learning models for vision"
ranked_docs = rank_documents(query, vectorizer, doc_term_matrix, top_n=3)
print(f"Top {len(ranked_docs)} documents for the query: '{query}'\n")
for rank, index, document, similarity in ranked_docs:
    print(f"Rank {rank}: (the {index}th document) {document} | Score = {similarity:.4f}")


Top 3 documents for the query: 'deep learning models for vision'

Rank 1: (the 3th document) Deep learning models like convolutional neural networks are powerful for computer vision tasks. | Score = 0.5550
Rank 2: (the 17th document) Hyperparameter tuning can significantly improve the accuracy of deep learning models. | Score = 0.3276
Rank 3: (the 0th document) The field of machine learning has seen rapid growth in recent years, especially in deep learning. | Score = 0.2139


### Task 3 Result
Top 3 Documents for the Query: *"deep learning models for vision"*

| Rank | Document Index | Content                                                                 | Score  |
|------|----------------|-------------------------------------------------------------------------|--------|
| 1    | 3              | Deep learning models like convolutional neural networks are powerful for computer vision tasks. | 0.5550 |
| 2    | 17             | Hyperparameter tuning can significantly improve the accuracy of deep learning models. | 0.3276 |
| 3    | 0              | The field of machine learning has seen rapid growth in recent years, especially in deep learning. | 0.2139 |


### Task 4: Implementing Viterbi for HMM POS Tagging (20 Marks)

**Objective:** Implement a simple Hidden Markov Model (HMM) POS tagger via the Viterbi algorithm.

**Description:** Implement Viterbi decoding for a small HMM and apply it to two sentences with the ambiguous word "book". Then briefly discuss why HMMs work for POS tagging and a limitation of the Markov assumption.

**Your task is to:**

1. Define two sentences:
    * `sentence1 = "The book is on the table."`
    * `sentence2 = "I want to book a flight."`
2. Implement Viterbi in log-space for a small tag set (e.g., `{DET, NOUN, VERB, PRT}`). Use the example initial (π), transition (A), and emission (B) probabilities in the parameters block below, or define your own consistent matrices and document them.
3. Run your decoder on both sentences and print the predicted tag sequence and total log-probability.
4. In a markdown cell, explain:
    * **a)** How transition and emission probabilities lead to different tags for "book" in the two sentences.
    * **b)** One sentence where the first-order Markov assumption is limiting, and why.

**Parameters:** Use the matrices shown in the section “Viterbi decoding for a simple HMM (Task 4)” below.

#### Viterbi decoding for a simple HMM (Task 4)

We illustrate HMM POS tagging with a small tag set `T = {DET, NOUN, VERB, PRT}` and vocabulary `V = {the, a, book, table, flight, is, want, to, on, i}`. The HMM comprises initial probabilities π, tag-to-tag transitions A, and tag-to-word emissions B.

Example parameters (each row sums to 1):

- Initial π:
  - P(DET)=0.50, P(NOUN)=0.20, P(VERB)=0.20, P(PRT)=0.10

- Transition A (rows: from-tag, cols: to-tag) in order [DET, NOUN, VERB, PRT]:

```text
from\to   DET    NOUN   VERB   PRT
DET      0.05   0.75   0.15   0.05
NOUN     0.05   0.10   0.75   0.10
VERB     0.10   0.35   0.40   0.15
PRT      0.05   0.10   0.75   0.10
```

- Emission B:
  - DET: the(0.80), a(0.20)
  - NOUN: book(0.45), table(0.25), flight(0.20), i(0.05), on(0.05)
  - VERB: is(0.40), want(0.35), book(0.20), to(0.03), on(0.02)
  - PRT: to(0.70), on(0.30)

Viterbi recurrence in log-space to avoid underflow:

- Initialization: `V[tag, 0] = log π[tag] + log B[tag, x0]`
- Recurrence: `V[tag, i] = log B[tag, xi] + max_prev ( V[prev, i-1] + log A[prev->tag] )`
- Backtrace from the best final tag.

We will decode the most likely tag sequence for the two Task 4 sentences using these parameters.


In [1]:
# Task 4: Implement Viterbi for a simple POS HMM (skeleton)
import math
from typing import List, Dict, Tuple

# Define your tag set
TAGS = ["DET", "NOUN", "VERB", "PRT"]

# Define HMM parameters (fill using the given parameters)
pi: Dict[str, float] = {
    "DET": 0.50,
    "NOUN": 0.20,
    "VERB": 0.20,
    "PRT": 0.10
}

A: Dict[str, Dict[str, float]] = {
    "DET":  {"DET": 0.05, "NOUN": 0.75, "VERB": 0.15, "PRT": 0.05},
    "NOUN": {"DET": 0.05, "NOUN": 0.10, "VERB": 0.75, "PRT": 0.10},
    "VERB": {"DET": 0.10, "NOUN": 0.35, "VERB": 0.40, "PRT": 0.15},
    "PRT":  {"DET": 0.05, "NOUN": 0.10, "VERB": 0.75, "PRT": 0.10}
}

B: Dict[str, Dict[str, float]] = {
    "DET":  {"the": 0.80, "a": 0.20},
    "NOUN": {"book": 0.45, "table": 0.25, "flight": 0.20, "i": 0.05, "on": 0.05},
    "VERB": {"is": 0.40, "want": 0.35, "book": 0.20, "to": 0.03, "on": 0.02},
    "PRT":  {"to": 0.70, "on": 0.30}
}

UNK = 1e-8

# Return log-probability for emitting 'word' from 'tag'. Use UNK for unseen words.
def emission_logprob(tag: str, word: str) -> float:
    prob = B.get(tag, {}).get(word, UNK)
    return math.log(prob)

# Implement Viterbi in log-space
# 1) initialize  2) dynamic programming with backpointers  3) termination + backtrace
def viterbi(tokens: List[str]) -> Tuple[List[str], float]:
    n = len(tokens)
    V = [{} for _ in range(n)]   # DP table
    backpointer = [{} for _ in range(n)]

    # Initialization
    for tag in TAGS:
        if pi[tag] > 0:
            V[0][tag] = math.log(pi[tag]) + emission_logprob(tag, tokens[0])
        else:
            V[0][tag] = -math.inf
        backpointer[0][tag] = None

    # Recurrence
    for i in range(1, n):
        for tag in TAGS:
            max_prob, best_prev = -math.inf, None
            for prev in TAGS:
                if V[i-1][prev] == -math.inf:
                    continue
                prob = V[i-1][prev] + math.log(A[prev][tag]) + emission_logprob(tag, tokens[i])
                if prob > max_prob:
                    max_prob, best_prev = prob, prev
            V[i][tag] = max_prob
            backpointer[i][tag] = best_prev

    # Termination
    final_tag = max(V[n-1], key=V[n-1].get)
    best_logprob = V[n-1][final_tag]

    # Backtrace
    tags = [final_tag]
    for i in range(n-1, 0, -1):
        tags.insert(0, backpointer[i][tags[0]])

    return tags, best_logprob

# Prepare the two sentences (lowercased tokens)
sentence1 = ["the", "book", "is", "on", "the", "table"]
sentence2 = ["i", "want", "to", "book", "a", "flight"]

# Run your decoder and print outputs
tags1, logp1 = viterbi(sentence1)
print(list(zip(sentence1, tags1)), " | logP=", round(logp1, 3))
tags2, logp2 = viterbi(sentence2)
print(list(zip(sentence2, tags2)), " | logP=", round(logp2, 3))


[('the', 'DET'), ('book', 'NOUN'), ('is', 'VERB'), ('on', 'PRT'), ('the', 'DET'), ('table', 'NOUN')]  | logP= -11.2
[('i', 'NOUN'), ('want', 'VERB'), ('to', 'PRT'), ('book', 'VERB'), ('a', 'DET'), ('flight', 'NOUN')]  | logP= -15.903


### Task 4 Result
#### Sentence 1: *"The book is on the table."*
| Word   | Tag  |
|--------|------|
| the    | DET  |
| book   | NOUN |
| is     | VERB |
| on     | PRT  |
| the    | DET  |
| table  | NOUN |

**Total log-probability:** `-11.2`

---

#### Sentence 2: *"I want to book a flight."*
| Word   | Tag  |
|--------|------|
| i      | NOUN |
| want   | VERB |
| to     | PRT  |
| book   | VERB |
| a      | DET  |
| flight | NOUN |

**Total log-probability:** `-15.903`


### **Analysis for Task 4**

* How transition and emission probabilities lead to different tags for "book" in the two sentences.


* One sentence where the first-order Markov assumption is limiting, and why.

In sentence 1: 'book' follows 'the', which is a DET. Tansition probility from 'DET' to 'NOUN' is 0.75, much higher than other cases. Emission probability P(book|none) is 0.45, also quite high.As a result, Viterbi decoding will choose 'NOUN' for 'book'.

In sentence 2:'book' follows 'to', which is a PRT. Tansition probility from 'PRT' to 'VERB' is high of 0.75. Emission probability P(book|verb) is 0.20, also kind of reasonable. As a result, Viterbi decoding will choose 'VERB' for 'book'.

Example sentence: **"The old man the boats."**
In this sentence, 'The old' means the old people, 'man' is a VERB representing manage. However, if the model of first-order Markov only consider one word above of 'old', it would consider 'old' as a ADJ, then predict 'man' as a NOUN. However, if taking the whole context into account, 'man' should be a VERB.
To sum up, the example shows that the model of first-order Markov is not enough to capture the long context and dependence between words. So the model may make mistakes when encountering long ambiguous context and complex dependencies. In a more comprehensive task, models like LSTM or Transformer can be used to capture long-term dependencies and context.



### Task 5: Comparing Cosine Similarity and Euclidean Distance (20 Marks)

**Objective:** To empirically demonstrate the difference between angle-based (Cosine) and magnitude-based (Euclidean) similarity measures in a vector space.

**Description:** The choice of similarity metric is crucial. This task highlights how document length affects each metric and why Cosine Similarity is often preferred for text-based topic similarity.

**Your task is to:**

1.  Define three simple documents:
    * `doc_A = "The cat sat on the mat."`
    * `doc_B = "The cat sat on the mat. The dog chased the cat."` (Longer, but on the same topic)
    * `doc_C = "The rocket launched into space."` (Different topic)
2.  Use `sklearn.feature_extraction.text.CountVectorizer` to transform these three documents into count vectors.
3.  Calculate the **Cosine Similarity** between all unique pairs of documents (A-B, A-C, B-C).
4.  Calculate the **Euclidean Distance** between all unique pairs of documents.
5.  Display your results clearly, for instance, in a Pandas DataFrame.
6.  **In a markdown cell, analyze your results:**
    * Explain why the Cosine Similarity between `doc_A` and `doc_B` is high, while their Euclidean Distance is relatively large.
    * Which metric (Cosine Similarity or Euclidean Distance) do your results suggest is better for identifying documents with similar topics, regardless of their length? Justify your answer based on your calculations.

In [4]:
# Your code for Task 5 here
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import pandas as pd

# Define documents
doc_A = "The cat sat on the mat."
doc_B = "The cat sat on the mat. The dog chased the cat."
doc_C = "The rocket launched into space."
corpus = [doc_A, doc_B, doc_C]

# Vectorize the documents
vectorizer = CountVectorizer()
text = vectorizer.fit_transform(corpus)

# Calculate similarities and distances
cos_sim = cosine_similarity(text)
euclid_distance = euclidean_distances(text)

# Display results in a DataFrame
cos_df = pd.DataFrame(cos_sim, index=["doc_A", "doc_B", "doc_C"], columns=["doc_A", "doc_B", "doc_C"])
euclid_df = pd.DataFrame(euclid_distance, index=["doc_A", "doc_B", "doc_C"], columns=["doc_A", "doc_B", "doc_C"])

print("Cosine Similarity Matrix:")
print(cos_df)
print("\nEuclidean Distance Matrix:")
print(euclid_df)

Cosine Similarity Matrix:
          doc_A     doc_B     doc_C
doc_A  1.000000  0.919239  0.316228
doc_B  0.919239  1.000000  0.357771
doc_C  0.316228  0.357771  1.000000

Euclidean Distance Matrix:
          doc_A     doc_B     doc_C
doc_A  0.000000  2.645751  3.000000
doc_B  2.645751  0.000000  4.690416
doc_C  3.000000  4.690416  0.000000


### Task 5 Result
#### Cosine Similarity Matrix

|       | doc_A   | doc_B   | doc_C   |
|-------|---------|---------|---------|
| **doc_A** | 1.000000 | 0.919239 | 0.316228 |
| **doc_B** | 0.919239 | 1.000000 | 0.357771 |
| **doc_C** | 0.316228 | 0.357771 | 1.000000 |

---

#### Euclidean Distance Matrix

|       | doc_A   | doc_B   | doc_C   |
|-------|---------|---------|---------|
| **doc_A** | 0.000000 | 2.645751 | 3.000000 |
| **doc_B** | 2.645751 | 0.000000 | 4.690416 |
| **doc_C** | 3.000000 | 4.690416 | 0.000000 |


### **Analysis for Task 5**

* Explain why the Cosine Similarity between `doc_A` and `doc_B` is high, while their Euclidean Distance is relatively large.

* Which metric (Cosine Similarity or Euclidean Distance) do your results suggest is better for identifying documents with similar topics, regardless of their length? Justify your answer based on your calculations.

Cosine Similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. It is often used to measure document similarity in text analysis.
In this case, the Cosine Similarity between `doc_A` and `doc_B` is high of `0.919` because they share a lot of common words, such as "the", "cat", and "on". The similarity is high because the angle between the two vectors is small, which means they are similar in direction.

Euclidean distance is a measure of the straight line distance between two points in Euclidean space. `doc_A` and `doc_B` Euclidean distance is large of `2.646` just because len(doc_B) is larger than len(doc_A), which makes the Euclidean distance larger.

So as the result shows, **Cosine Similarity** is better for identifying documents with similar topics only focusing on the word distribution similarity, regardless of their length. Euclidean distance may misinterpret the length of the document as a factor, which may not be the case in many practical applications.